# Hands-on: Deciphering Inka Khipu Signs

In [17]:
%matplotlib inline
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt

In [3]:
def plot_multi_proportion_confint_digits(actual_first_digit_frequencies, 
                                         actual_first_digit_counts, 
                                         expected_first_digit_frequencies,
                                         label_actual,
                                         label_expected,
                                         title
                                        ):
    # Import function-specific package: return array of lower and upper confidence levels for each category
    from statsmodels.stats.proportion import multinomial_proportions_confint
    
    # Plot 95% CIs as lines with caps on the top and bottom for easier interpretation
    plt.figure(figsize=(9,5))

    (_, caps, _) = plt.errorbar(x=np.arange(1, 10, 1), 
                     y=actual_first_digit_frequencies, 
                     yerr=[(top-bot)/2 for top,bot in multinomial_proportions_confint(actual_first_digit_counts)],
                     capsize=5, fmt='none', label=label_actual+' 95% CI')

    for cap in caps:
        cap.set_markeredgewidth(1)

    # Plot expected values as scatter plot points, in order to visualize where CIs include expected values
    plt.scatter(np.arange(1,10,1), expected_first_digit_frequencies, label=label_expected, color='red')
    
    # Format 
    plt.xticks([1,2,3,4,5,6,7,8,9])
    plt.xlabel('First Digits')
    plt.ylabel('Frequency')
    plt.title(title)
    plt.legend();
    
    return

expected_frequencies = pd.Series([.301, .176, .125, .097, .079, .067, .058, .051, .046], 
                                 index = [1, 2, 3, 4, 5, 6, 7, 8, 9]
                                )

In [26]:
khipu_data = pd.read_pickle('khipu_cords.pkl')
    
khipu_data


,Unnamed: 0,Khipu,Cord,Fiber,Value,Colors,Spin_Ply,Attachment,Termin,Length,Anomalous
0,0,AB001,1,NaN,0,NaN,S,R,U,42.0,False
1,1,AB001,1,NaN,0,NaN,S,R,U,42.5,False
2,2,AB001,1,NaN,0,NaN,S,V,K,50.0,False
3,3,AB001,2,NaN,0,NaN,S,R,B,24.5,False
4,4,AB001,1,NaN,0,NaN,S,R,B,19.0,False
...,...,...,...,...,...,...,...,...,...,...,...
56822,3146,UR294,1,CN,100,LB,Z,V,K,34.5,False
56823,3147,UR294,2,CN,22,MB,Z,V,B,22.5,False
56824,3148,UR294,3,CN,65,NB,Z,V,K,17.5,False
56825,3149,UR294,4,CN,52,W,Z,V,K,23.0,False
